In [1]:
import os
import sys
import glob
import cv2
import numpy as np
from tqdm import tqdm
from medpy.io import save
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score, jaccard_similarity_score
from tqdm import tqdm

from fbs import FBSDataset, FBSConfig
  
from mrcnn import model as modellib    
from mrcnn.utils import extract_bboxes, non_max_suppression

from datahandler import DataHandler
  
ROOT_DIR = os.path.abspath('../../')
sys.path.append(ROOT_DIR)         
      
LOG_DIR = os.path.join(ROOT_DIR, 'logs')
MODEL_DIR = os.path.join(LOG_DIR, "mask_rcnn")
DATASET_DIR = os.path.join(ROOT_DIR, 'data/')
IMAGES_DIR = os.path.join(DATASET_DIR, 'test/images/*')
MASKS_DIR = os.path.join(DATASET_DIR, 'test/masks/*')

dh = DataHandler()

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12024568863552552414
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10112886375
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16712919891816202248
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [2]:
class InferenceConfig(FBSConfig):
    IMAGES_PER_GPU = 1
    MASK_SHAPE = [56,56]

inference_config = InferenceConfig()

model = modellib.MaskRCNN(        
         mode='inference',
         config=inference_config,
         model_dir=MODEL_DIR
         )      
  
model_path = '../../logs/mask_rcnn/fbs_resnet50_da_tf_nominimask_1channel20190319T1724/mask_rcnn_fbs_resnet50_da_tf_nominimask_1channel_0070.h5'
model.load_weights(model_path, by_name=True)

Re-starting from epoch 70


In [3]:
def destiny_directory(data_origin, dice_score):
    pre = './data/eval/mask_rcnn_'+data_origin+'/'
    if dice_score >= 98:
        return pre + 'dice_98_100/'
    elif dice_score >= 96:
        return pre + 'dice_96_98/'
    elif dice_score >= 94:
        return pre + 'dice_94_96/'
    elif dice_score >= 92:
        return pre + 'dice_92_94/'
    elif dice_score >= 90:
        return pre + 'dice_90_92/'
    elif dice_score >= 88:
        return pre + 'dice_88_90/'
    elif dice_score >= 85:
        return pre + 'dice_85_88'
    elif dice_score >= 80:
        return pre + 'dice_80_85/'
    elif dice_score >= 70:
        return pre + 'dice_70_80/'
    elif dice_score >= 60:
        return pre + 'dice_60_70/'
    else:
        return pre + 'dice_less_60'
    
def getFileName(fname):
    original_name = fname.split('/')[-1]
    original_name = original_name[:original_name.index('.')]
    return original_name

def evaluateMask(ground_truth, prediction):
    #convert to boolean values and flatten
    ground_truth = np.asarray(ground_truth, dtype=np.bool).flatten()
    prediction = np.asarray(prediction, dtype=np.bool).flatten()    
    return f1_score(ground_truth, prediction)

def saveAll(data_origin, fname, hdr, image, gt_mask, pred_mask, score):
    fname = getFileName(fname)
    dice_score = int(score * 100)
    
    save_path = destiny_directory(data_origin, dice_score)
    save_path = os.path.join(ROOT_DIR, save_path)
        
    save(pred_mask, os.path.join(save_path, fname + '_pred_' 
        + str(dice_score) + '.nii'), hdr)
    save(image, os.path.join(save_path, fname + '_img.nii'), hdr)
    save(gt_mask, os.path.join(save_path, fname + '_mask.nii'), hdr)

In [4]:
import itertools

def vflip(image):
    return np.flipud(image)

def hflip(image):
    return np.fliplr(image)

def rotate(image, k):
    return np.rot90(image, k, axes=(0,1))

def ensemble_prediction(model, image):
    """Test time augmentation method using non-maximum supression"""
    masks = []
    results = {}
    
    result = model.detect([image], verbose=0)[0]
    
    if result['masks'].shape[2] == 0:
        return result
    
    masks.append(result['masks'])
    
    flip_v = [True, False]
    flip_h = [True, False]
    rotations = [0,1,2,3]

    transformations = list(itertools.product(flip_v, flip_h, rotations))

    for fv, fh, r in transformations:
        result = image
        result = vflip(result) if fv else result
        result = hflip(result) if fh else result
        result = rotate(result, r)
        result = model.detect([result], verbose=0)[0]['masks']
        result = rotate(result, -r)
        result = hflip(result) if fh else result
        result = vflip(result) if fv else result
        masks.append(result)
        
    masks = np.concatenate(masks, axis=-1)
    masks = np.sum(masks, axis=-1, keepdims=True)
    
    cut_off = len(transformations) // 2
    masks[masks <= cut_off] == 0
    masks[masks > cut_off] == 1
    results['masks'] = masks

    return results

In [ ]:
def predictAll(data_origin, dict_prefix, tta=False, dilate=False, d_size=None):
    data_origin+=dict_prefix
    
    image_paths = glob.glob(IMAGES_DIR)
    masks_paths = glob.glob(MASKS_DIR)
    dice_scores = []
    
    if dilate:
        kernel = np.ones(d_size,np.uint8)

    for image_path, mask_path in tqdm(zip(image_paths, masks_paths),
                                      total=len(image_paths)):
        #get header of image to later save mask
        full_image, hdr = dh.getImageData(image_path)
        gt_mask, _ = dh.getImageData(mask_path, is_mask=True)

        dataset_val = FBSDataset()        
        dataset_val.load_data(DATASET_DIR, 
                              subset='eval', 
                              image_file=image_path, 
                              mask_file=mask_path)
        dataset_val.prepare() 
    
        prediction = []
        
        for img_id in dataset_val.image_ids:
            image, image_meta, class_ids, bbox, mask = modellib.load_image_gt(
            dataset_val, inference_config, img_id, use_mini_mask=False)

            if tta:
                results = ensemble_prediction(model, image)
                r = results
                
            else:
                results = model.detect([image], verbose = 0)
                r = results[0]
                
            pred = r['masks']

            if(len(pred.shape) > 2 and pred.shape[2] == 0):
                pred = np.zeros((256,256,1))
            
            if dilate:
                pred = np.asarray(pred, dtype=np.uint8)
                pred = cv2.dilate(pred,kernel,iterations = 1)

            prediction.append(pred)

        pred_mask = np.asarray(prediction, dtype=np.bool)

        score = evaluateMask(gt_mask, pred_mask)
        dice_scores.append(score)
        
        saveAll(data_origin, image_path, hdr, full_image, 
                gt_mask, pred_mask, score)
        
    
    print('Number of images %d'%len(dice_scores))
    print(np.mean(dice_scores))

In [ ]:
data_origin = 'test'
dict_prefix = ''
predictAll(data_origin, dict_prefix)

100%|██████████| 43/43 [04:48<00:00,  6.83s/it]

Number of images 43
0.9056250096248001


In [ ]:
data_origin = 'test'
dict_prefix = 'TTA'
predictAll(data_origin, dict_prefix, tta=True)

 30%|███       | 13/43 [11:44<27:16, 54.56s/it]

In [ ]:
data_origin = 'test'
dict_prefix = 'Dilate3'
predictAll(data_origin, dict_prefix, dilate=True, d_size=(3,3))

In [ ]:
data_origin = 'test'
dict_prefix = 'Dilate5'
predictAll(data_origin, dict_prefix, dilate=True, d_size=(5,5))